<a href="https://colab.research.google.com/github/ernesto-butto/ag2-workshop-simulation/blob/main/gameMasterAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro


Instalar AG2
https://docs.ag2.ai/docs/home/quick-start

In [11]:
!pip install ag2

# Setup

In [12]:
# Surpress warnings
import warnings
warnings.filterwarnings("ignore")


In [13]:
# 1. Import our agent class
from autogen import ConversableAgent, UserProxyAgent
from pprint import pprint

In [14]:
llm_config = {"config_list": [{
    "model": "gpt-4o-mini",  # This should match your Azure deployment name
    "api_key": "0d20814cba1242d39825274ccd99d2e5",
    "base_url": "https://ai-ernestobuttoai916354951815.services.ai.azure.com/",
    "api_type": "azure",
    "api_version": "2024-08-01-preview",
    "temperature": 0.8
}]}

# Agente Creador de Escenarios
🔹 **Attendees Build:**

- **(A) Scenario Creation Agent** → Generates a simulation scenario.
- **(B) Mocked User Agent** → Simulates a user providing input for scenario generation.


In [15]:
fin_de_conversacion_keyword = "FIN_CONVERSACION"

In [16]:
def createGameMasterAgent(escenario):

  return ConversableAgent(
      name="gameMasterAgent",
      llm_config=llm_config,
      code_execution_config=False,
      human_input_mode="NEVER",
      system_message=f"""
      **Meta**: Este es un escenario de juego de roles en el que el usuario (estudiante) practica alguna habilidad que le interese mejorar

      **Persona**: En este escenario interpretas AI Mentor, un amigable y práctico mentor

      **Narrativa**: Dado el escenario de abajo, interpreta la contrapartida del usuario.

      EMPIEZA_EL_ESCENARIO
      {escenario}
      TERMINA_EL_ESCENARIO

      **Debes hacer lo siguiente**
      1. Interpreta la contrapartida del usuario
      2. Después de 6 turnos, intenta de cerrar la interacción
      3. Si el estudiante lo está haciendo bien, considera aumentar la dificultad de la simulación, haciendola más retadora
      4. Marca el final de la conversación con el keyword: {fin_de_conversacion_keyword}

      """,
      is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"],
  )


In [17]:
def mockedUserAgent():
  return ConversableAgent(
        "mocked_student",
        llm_config=llm_config,
        human_input_mode="NEVER",  # always ask for human input
        system_message=f"""
        Eres un estudiante que está aprendiendo habilidades atravéz de simulaciones,
        vas a hacer una simulación con una pareja de una habilidad que estás aprendiendo todavía, y necesitas practicar para mejorar.
        """,
        code_execution_config=False,
        is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"],
    )

# Simulacion estudiante simulado

In [21]:
scenario = f"""
**Escenario: Apoyo en la Playa de Surf**

Ubicación: Una playa popular para surfistas, con olas suaves y un ambiente relajado.

Objetivo: Acercarte a un surfista que parece desanimado y practicar tu empatía al escuchar sus preocupaciones y ofrecerle apoyo.

Descripción:
Te encuentras en la playa, disfrutando de un día de surf. Observas a un surfista que está sentado en la arena, mirando al mar con expresión de frustración. Parece que ha estado intentando atrapar olas, pero no ha tenido suerte.

Tu tarea es acercarte a él, saludarlo y preguntar cómo se siente respecto al surf ese día. A medida que conversas, escucha atentamente lo que dice, valida sus sentimientos y compárteles alguna experiencia similar que hayas tenido, para mostrarle que comprendes lo que está pasando.

Al final de la conversación, intenta animarlo y sugiere que podrían intentar surfear juntos para que se sienta más apoyado.

¡Espero que disfrutes mucho practicando este escenario! Si necesitas más ayuda en el futuro, aquí estaré.

FIN_ESCENARIO

"""

In [22]:
# Formatting Break long lines
from IPython.display import HTML, display

display(HTML("""
<style>
pre {
    white-space: pre-wrap;       /* css-3 */
    white-space: -moz-pre-wrap;  /* Mozilla, since 1999 */
    white-space: -pre-wrap;      /* Opera 4-6 */
    white-space: -o-pre-wrap;    /* Opera 7 */
    word-wrap: break-word;       /* Internet Explorer 5.5+ */
}
</style>
"""))

# Crear el Game Master
game_master = createGameMasterAgent(scenario)

# Crear el estudiante simulado
estudiante_simulado = mockedUserAgent()
chat_result = estudiante_simulado.initiate_chat(
    game_master,
    message="Hola!",
    max_turns=12,
    summary_method="reflection_with_llm"
)

mocked_student (to gameMasterAgent):

Hola!

--------------------------------------------------------------------------------
gameMasterAgent (to mocked_student):

¡Hola! Veo que estás disfrutando de la playa, ¿cómo te sientes hoy con el surf? Te noto un poco pensativo.

--------------------------------------------------------------------------------
mocked_student (to gameMasterAgent):

Hola, sí, he estado pensando en cómo mejorar mis habilidades de surf. Me encanta estar en el agua, pero a veces me cuesta mantener el equilibrio y atrapar las olas. ¿Tienes algún consejo o técnica que me pueda ayudar?

--------------------------------------------------------------------------------
gameMasterAgent (to mocked_student):

Entiendo cómo te sientes. Mejorar en el surf puede ser frustrante a veces. También he pasado por momentos en los que simplemente no podía atrapar una ola, y eso puede ser desalentador. Pero lo importante es recordar que todos pasamos por altibajos. 

Una técnica que me a

In [23]:
print("RESUMEN")
pprint(chat_result.summary)


print("COSTO")
pprint(chat_result.cost)
print()

RESUMEN
('The conversation focused on improving surfing skills, specifically balance '
 'and technique for standing up on the board. Key takeaways include practicing '
 'the correct posture, breathing to manage nerves, and the importance of '
 'visualization and support from a partner. Consistent practice on land helped '
 'build confidence for surfing in the water. Overall, maintaining a positive '
 'attitude and enjoying the process were emphasized as essential for '
 'improvement.')
COSTO
{'usage_excluding_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 1638,
                                                                 'cost': 0.0038477999999999997,
                                                                 'prompt_tokens': 19100,
                                                                 'total_tokens': 20738},
                                      'total_cost': 0.0038477999999999997},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18

# Interacción con usuario real


In [ ]:
real_user_parnter = UserProxyAgent(
            name="user",
            human_input_mode="ALWAYS",
        )

In [ ]:
# chat_result = creador_de_escenario.initiate_chat(
#     real_user_parnter,
#     message="Hola!",
#     max_turns=12,
#     summary_method="reflection_with_llm",
#     summary_prompt="Escribe el escenario en Español",
#     is_termination_msg=lambda msg: fin_de_conversacion_keyword in msg["content"]
# )

In [ ]:
print("SUMMARY")
pprint(chat_result.summary)
print()

print("COST")
pprint(chat_result.cost)
print()

pprint("ESCENARIO")
escenario_2 = chat_result.chat_history[-1]["content"]
print(escenario_2)
print()
